In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telco-customer-churn/TelcoCustomerChurn.csv


# Purpose of Customer Churn Analysis

**Objective:** The primary goal of customer churn analysis is to understand the factors contributing to customer attrition and to identify trends and patterns that can help reduce churn rates.

**Business Impact:** High churn rates can significantly impact a company's revenue and profitability. By analyzing churn, businesses can develop targeted strategies to improve customer retention, enhance customer satisfaction, and ultimately increase lifetime value.

**Strategic Use:** Insights from churn analysis enable businesses to proactively address issues, optimize customer service, tailor marketing efforts, and design more compelling customer experiences.

In [2]:
import pandas as pd
import sqlite3

# Load the CSV file into a DataFrame
file_path = '/kaggle/input/telco-customer-churn/TelcoCustomerChurn.csv'  # Adjust the path if needed
telco_data = pd.read_csv(file_path)

# Connect to SQLite (it will create a new database file if it doesn't exist)
conn = sqlite3.connect('telco_customer_churn.db')  # You can specify a different path or name

# Load the DataFrame into a SQL table
telco_data.to_sql('TelcoCustomerChurn', conn, if_exists='replace', index=False)

# Now you can run SQL queries on the SQLite database
cursor = conn.cursor()

# Check for duplicates

Analyzes the dataset to identify duplicate customer entries, ensuring data integrity and accuracy in churn analysis.

In [3]:
# 1. Check for Duplicates
check_duplicates_query = """
SELECT CustomerID, COUNT(*) AS DuplicateCount
FROM TelcoCustomerChurn
GROUP BY CustomerID
HAVING COUNT(*) > 1
"""
check_duplicates_result = pd.read_sql_query(check_duplicates_query, conn)
check_duplicates_result

,CustomerID,DuplicateCount


# Total number of customers

- This query retrieves the total count of customers in the TelcoCustomerChurn dataset.

In [4]:
# 2. Total Number of Customers
total_customers_query = """
SELECT COUNT(*) AS TotalCustomers
FROM TelcoCustomerChurn
"""
total_customers_result = pd.read_sql_query(total_customers_query, conn)
total_customers_result

,TotalCustomers
0,7043


# Total number of customers (excluding recently joined)

Counts the total number of customers in the dataset while excluding those who have recently joined, focusing on longer-term customers for churn analysis.

In [5]:
# 3. Total Number of Customers Who Have Stayed or Churned (Q3)
total_customers_stayed_churned_query = """
SELECT COUNT(*) AS TotalCustomers
FROM TelcoCustomerChurn
WHERE CustomerStatus <> 'Joined'
"""
total_customers_stayed_churned_result = pd.read_sql_query(total_customers_stayed_churned_query, conn)
total_customers_stayed_churned_result

,TotalCustomers
0,6589


# Total number of customers who didn't churn

Calculates the total number of customers who have not churned, providing a clear picture of customer retention.

In [6]:
# 4. Total Remaining Customers (Not Churned)
total_remaining_customers_query = """
SELECT COUNT(*) AS RemainingCustomers
FROM TelcoCustomerChurn
WHERE CustomerStatus <> 'Churned'
"""
total_remaining_customers_result = pd.read_sql_query(total_remaining_customers_query, conn)
total_remaining_customers_result

,RemainingCustomers
0,5174


# Overall churn rate

Computes the overall churn rate for the dataset, indicating the percentage of customers that have left the service over a specified period.

In [7]:
# 5. Overall Churn Rate in Q3
overall_churn_rate_query = """
SELECT COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / (SELECT COUNT(*) FROM TelcoCustomerChurn WHERE CustomerStatus <> 'Joined')) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
"""
overall_churn_rate_result = pd.read_sql_query(overall_churn_rate_query, conn)
overall_churn_rate_result

,ChurnCounts,ChurnRate
0,1869,28.365458


# Calculate churn rate by gender

This query evaluates the churn rate segmented by customer gender, offering insights into whether gender influences retention.

In [8]:
# 6. Churn Rate by Gender for Q3
churn_rate_by_gender_query = """
SELECT Gender, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Gender
ORDER BY ChurnRate DESC
"""
churn_rate_by_gender_result = pd.read_sql_query(churn_rate_by_gender_query, conn)
churn_rate_by_gender_result

,Gender,ChurnCounts,ChurnRate
0,Female,939,50.24077
1,Male,930,49.75923


# Calculate churn rate by Age

Calculates the distribution of customer ages, providing insights into the demographic profile of the customer base.

In [9]:
# 7. Churn Rate by Age for Q3
churn_rate_by_age_query = """
SELECT Age, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Age
ORDER BY ChurnRate DESC, Age
"""
churn_rate_by_age_result = pd.read_sql_query(churn_rate_by_age_query, conn)
churn_rate_by_age_result

,Age,ChurnCounts,ChurnRate
0,44,41,2.193686
1,47,41,2.193686
2,53,41,2.193686
3,68,40,2.140182
4,39,39,2.086677
...,...,...,...
57,43,22,1.177100
58,45,20,1.070091
59,19,18,0.963082
60,24,18,0.963082


# Churn rate by age group

Segments customers by age groups and calculates the churn rate within these segments, providing insights into age-related retention trends.

In [10]:
# 8. Churn Rate by Age Groups for Q3
churn_rate_by_age_groups_query = """
WITH Churned AS (
    SELECT 
        CASE
            WHEN Age BETWEEN 18 AND 39 THEN 'Young Adult (18-39)'
            WHEN Age BETWEEN 40 AND 59 THEN 'Middle-Aged (40-59)'
            WHEN Age BETWEEN 60 AND 100 THEN 'Senior (60+)'
        END AS AgeGroup, COUNT(*) AS ChurnCounts
    FROM TelcoCustomerChurn
    WHERE CustomerStatus = 'Churned'
    GROUP BY 
        CASE
            WHEN Age BETWEEN 18 AND 39 THEN 'Young Adult (18-39)'
            WHEN Age BETWEEN 40 AND 59 THEN 'Middle-Aged (40-59)'
            WHEN Age BETWEEN 60 AND 100 THEN 'Senior (60+)'
        END
)
SELECT AgeGroup, ChurnCounts, 
       (ChurnCounts * 1.0 / SUM(ChurnCounts) OVER ()) * 100 AS ChurnRate  
FROM Churned
ORDER BY ChurnRate DESC
"""
churn_rate_by_age_groups_result = pd.read_sql_query(churn_rate_by_age_groups_query, conn)
churn_rate_by_age_groups_result

,AgeGroup,ChurnCounts,ChurnRate
0,Senior (60+),631,33.761370
1,Middle-Aged (40-59),625,33.440342
2,Young Adult (18-39),613,32.798288


# Churn rate by age and gender

Analyzes churn rates considering both age and gender, helping to identify if specific demographics are more prone to churn.

In [11]:
# 9. Churn Rate by Age and Gender for Q3
churn_rate_by_age_gender_query = """
SELECT Age, Gender, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Age, Gender
ORDER BY ChurnRate DESC
"""
churn_rate_by_age_gender_result = pd.read_sql_query(churn_rate_by_age_gender_query, conn)
churn_rate_by_age_gender_result

,Age,Gender,ChurnCounts,ChurnRate
0,68,Female,25,1.337614
1,44,Male,24,1.284109
2,47,Female,22,1.177100
3,71,Male,22,1.177100
4,30,Male,21,1.123596
...,...,...,...,...
119,24,Female,8,0.428036
120,45,Male,7,0.374532
121,19,Male,6,0.321027
122,71,Female,5,0.267523


# Churn rate by age group and gender

Segments customers into age groups and genders to calculate churn rates, revealing more nuanced demographic trends in customer retention.

In [12]:
# 10. Churn Rate by Age Groups and Gender for Q3
churn_rate_by_age_gender_groups_query = """
WITH Churned AS (
    SELECT 
        CASE
            WHEN Age BETWEEN 18 AND 39 THEN 'Young Adult (18-39)'
            WHEN Age BETWEEN 40 AND 59 THEN 'Middle-Aged (40-59)'
            WHEN Age BETWEEN 60 AND 100 THEN 'Senior (60+)'
        END AS AgeGroup, Gender, COUNT(*) AS ChurnCounts
    FROM TelcoCustomerChurn
    WHERE CustomerStatus = 'Churned'
    GROUP BY 
        CASE
            WHEN Age BETWEEN 18 AND 39 THEN 'Young Adult (18-39)'
            WHEN Age BETWEEN 40 AND 59 THEN 'Middle-Aged (40-59)'
            WHEN Age BETWEEN 60 AND 100 THEN 'Senior (60+)'
        END, Gender
)
SELECT AgeGroup, Gender, ChurnCounts, 
       (ChurnCounts * 1.0 / SUM(ChurnCounts) OVER ()) * 100 AS ChurnRate  
FROM Churned
ORDER BY ChurnRate DESC
"""
churn_rate_by_age_gender_groups_result = pd.read_sql_query(churn_rate_by_age_gender_groups_query, conn)
churn_rate_by_age_gender_groups_result

,AgeGroup,Gender,ChurnCounts,ChurnRate
0,Senior (60+),Male,320,17.121455
1,Young Adult (18-39),Female,316,16.907437
2,Middle-Aged (40-59),Male,313,16.746923
3,Middle-Aged (40-59),Female,312,16.693419
4,Senior (60+),Female,311,16.639914
5,Young Adult (18-39),Male,297,15.890851


# Calculate churn rate by marital status

This query assesses the churn rate based on the marital status of customers, which may indicate how personal circumstances affect customer loyalty.

In [13]:
# 11. Churn Rate Based on Marital Status for Q3
churn_rate_by_marital_status_query = """
SELECT Married, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Married
ORDER BY ChurnRate DESC
"""
churn_rate_by_marital_status_result = pd.read_sql_query(churn_rate_by_marital_status_query, conn)
churn_rate_by_marital_status_result

,Married,ChurnCounts,ChurnRate
0,No,1200,64.205457
1,Yes,669,35.794543


# Churn rate by dependents

Evaluates churn rates based on whether customers have dependents, providing insights into how family status influences customer loyalty.

In [14]:
# 12. Churn Rate Based on Dependent Status for Q3
churn_rate_by_dependent_status_query = """
SELECT Dependents, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Dependents
ORDER BY ChurnRate DESC
"""
churn_rate_by_dependent_status_result = pd.read_sql_query(churn_rate_by_dependent_status_query, conn)
churn_rate_by_dependent_status_result

,Dependents,ChurnCounts,ChurnRate
0,No,1763,94.328518
1,Yes,106,5.671482


# Churn rate by Number of dependents

Retrieves the total count of dependents associated with customers, helping to understand family structures within the customer base.

In [15]:
# 13. Churn Rate by Number of Dependents for Q3
churn_rate_by_number_of_dependents_query = """
SELECT NumberofDependents, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY NumberofDependents
ORDER BY ChurnRate DESC
"""
churn_rate_by_number_of_dependents_result = pd.read_sql_query(churn_rate_by_number_of_dependents_query, conn)
churn_rate_by_number_of_dependents_result

,NumberofDependents,ChurnCounts,ChurnRate
0,0,1763,94.328518
1,1,38,2.033173
2,2,33,1.765650
3,3,30,1.605136
4,5,2,0.107009
5,4,1,0.053505
6,6,1,0.053505
7,7,1,0.053505


# Churn Rate by Gender, Marriage Status, and Number of Dependents

This query evaluates how the combination of gender, marital status, and number of dependents affects churn behavior. By grouping customers based on these demographic factors, it provides a nuanced understanding of which segments are more likely to churn. This insight can help in designing targeted retention strategies for specific customer profiles.

In [16]:
# 14. Churn Rate by Gender, Marriage Status, and Number of Dependents for Q3
churn_rate_by_gender_marriage_dependents_query = """
SELECT 
    Gender,
    Married,
    NumberofDependents,
    COUNT(*) AS ChurnCounts,
    (COUNT(*) * 1.0 / (SELECT COUNT(*) FROM TelcoCustomerChurn WHERE CustomerStatus = 'Churned')) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Gender, Married, NumberofDependents
ORDER BY ChurnRate DESC
"""
churn_rate_by_gender_marriage_dependents_result = pd.read_sql_query(churn_rate_by_gender_marriage_dependents_query, conn)
churn_rate_by_gender_marriage_dependents_result

,Gender,Married,NumberofDependents,ChurnCounts,ChurnRate
0,Female,No,0,594,31.781701
1,Male,No,0,556,29.748529
2,Male,Yes,0,328,17.549492
3,Female,Yes,0,285,15.248796
4,Female,No,2,12,0.642055
5,Male,Yes,1,12,0.642055
6,Female,Yes,1,11,0.588550
7,Female,Yes,2,11,0.588550
8,Female,Yes,3,11,0.588550
9,Male,No,3,9,0.481541


# Churn rate by city

Calculates churn rates segmented by city, helping to identify geographic trends and potential regional issues affecting retention.

In [17]:
# 15. Churn Rate by City for Q3
churn_rate_by_city_query = """
SELECT City, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY City
ORDER BY ChurnRate DESC
"""
churn_rate_by_city_result = pd.read_sql_query(churn_rate_by_city_query, conn)
churn_rate_by_city_result

,City,ChurnCounts,ChurnRate
0,San Diego,185,9.898341
1,Los Angeles,78,4.173355
2,San Francisco,31,1.658641
3,San Jose,29,1.551632
4,Fallbrook,26,1.391118
...,...,...,...
772,Woodland Hills,1,0.053505
773,Yermo,1,0.053505
774,Yorkville,1,0.053505
775,Yucaipa,1,0.053505


In [18]:
# 16. Churn Rate by City for Q3 (With Total Customers)
churn_rate_by_city_with_total_query = """
SELECT City, 
       COUNT(*) AS TotalCustomers, 
       COUNT(CASE WHEN CustomerStatus = 'Churned' THEN 1 END) AS ChurnCounts, 
       (COUNT(CASE WHEN CustomerStatus = 'Churned' THEN 1 END) * 1.0 / (SELECT COUNT(*) FROM TelcoCustomerChurn WHERE CustomerStatus = 'Churned')) * 100 AS ChurnRate
FROM TelcoCustomerChurn
GROUP BY City
ORDER BY ChurnRate DESC
"""
churn_rate_by_city_with_total_result = pd.read_sql_query(churn_rate_by_city_with_total_query, conn)
churn_rate_by_city_with_total_result

,City,TotalCustomers,ChurnCounts,ChurnRate
0,San Diego,285,185,9.898341
1,Los Angeles,293,78,4.173355
2,San Francisco,104,31,1.658641
3,San Jose,112,29,1.551632
4,Sacramento,108,26,1.391118
...,...,...,...,...
1101,Aliso Viejo,4,0,0.000000
1102,Albion,4,0,0.000000
1103,Alamo,4,0,0.000000
1104,Alameda,8,0,0.000000


# Churn rate by tenure

Calculates the churn rate segmented by customer tenure, helping to understand how long customers stay before churning.

In [19]:
# 17. Churn Rate by Tenure Years for Q3
churn_rate_by_tenure_years_query = """
SELECT 
    CASE 
        WHEN TenureinMonths BETWEEN 0 AND 12 THEN '0-1 year'
        WHEN TenureinMonths BETWEEN 13 AND 36 THEN '1-3 years'
        WHEN TenureinMonths BETWEEN 37 AND 60 THEN '3-5 years'
        ELSE '5+ years'
    END AS TenureRange,
    COUNT(*) AS ChurnCounts,
    (COUNT(*) * 1.0 / (SELECT COUNT(*) FROM TelcoCustomerChurn WHERE CustomerStatus = 'Churned')) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY 
    CASE 
        WHEN TenureinMonths BETWEEN 0 AND 12 THEN '0-1 year'
        WHEN TenureinMonths BETWEEN 13 AND 36 THEN '1-3 years'
        WHEN TenureinMonths BETWEEN 37 AND 60 THEN '3-5 years'
        ELSE '5+ years'
    END
ORDER BY ChurnRate DESC
"""
churn_rate_by_tenure_years_result = pd.read_sql_query(churn_rate_by_tenure_years_query, conn)
churn_rate_by_tenure_years_result

,TenureRange,ChurnCounts,ChurnRate
0,0-1 year,1037,55.484216
1,1-3 years,474,25.361156
2,3-5 years,265,14.178705
3,5+ years,93,4.975923


# Churn rate by received offer

Analyzes how receiving promotional offers impacts churn rates, revealing the effectiveness of marketing strategies on customer retention.

In [20]:
# 18. Churn Rate by Received Offer for Q3
churn_rate_by_received_offer_query = """
SELECT Offer, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Offer
ORDER BY ChurnRate DESC
"""
churn_rate_by_received_offer_result = pd.read_sql_query(churn_rate_by_received_offer_query, conn)
churn_rate_by_received_offer_result

,Offer,ChurnCounts,ChurnRate
0,None,1051,56.233280
1,Offer E,426,22.792937
2,Offer D,161,8.614232
3,Offer B,101,5.403959
4,Offer C,95,5.082932
5,Offer A,35,1.872659


# Calculate churn rate by phone service and multiple lines for Q3

This query calculates the churn rate grouped by the type of phone service and whether customers have multiple lines, providing insights into service features that may affect churn.

In [21]:
# 19. Churn Rate by Phone Service and Multiple Lines for Q3
churn_rate_by_phone_service_multiple_lines_query = """
SELECT PhoneService, MultipleLines, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY PhoneService, MultipleLines
ORDER BY ChurnRate DESC
"""
churn_rate_by_phone_service_multiple_lines_result = pd.read_sql_query(churn_rate_by_phone_service_multiple_lines_query, conn)
churn_rate_by_phone_service_multiple_lines_result

,PhoneService,MultipleLines,ChurnCounts,ChurnRate
0,Yes,Yes,850,45.478866
1,Yes,No,849,45.425361
2,No,No,170,9.095773


# Calculate churn rate by phone service and internet service for Q3

This query evaluates the churn rate based on the combination of phone service types and internet service, helping to identify which service pairings may have higher churn.

In [22]:
# 20. Churn Rate by Phone Service and Internet Service for Q3
churn_rate_by_phone_service_internet_service_query = """
SELECT PhoneService, InternetService, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY PhoneService, InternetService
ORDER BY ChurnRate DESC
"""
churn_rate_by_phone_service_internet_service_result = pd.read_sql_query(churn_rate_by_phone_service_internet_service_query, conn)
churn_rate_by_phone_service_internet_service_result

,PhoneService,InternetService,ChurnCounts,ChurnRate
0,Yes,Yes,1586,84.858213
1,No,Yes,170,9.095773
2,Yes,No,113,6.046014


# Calculate churn rate based on internet service types for Q3

This query analyzes the churn rate specifically for different types of internet services provided to customers, indicating potential issues with particular internet offerings.

In [23]:
# 21. Churn Rate Based on Internet Service Types for Q3
churn_rate_based_on_internet_service_query = """
SELECT InternetType, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY InternetType
ORDER BY ChurnRate DESC
"""
churn_rate_based_on_internet_service_result = pd.read_sql_query(churn_rate_based_on_internet_service_query, conn)
churn_rate_based_on_internet_service_result

,InternetType,ChurnCounts,ChurnRate
0,Fiber Optic,1236,66.131621
1,DSL,307,16.425896
2,Cable,213,11.396469
3,None,113,6.046014


# Churn rate by internet service, internet service types, average monthly GB download for Q3

This query calculates the churn rate segmented by internet service, internet type, and average monthly GB downloaded, which may highlight the impact of usage on retention.

In [24]:
# 22. Churn Rate by Internet Service, Internet Service Types, Average Monthly GB Download for Q3
churn_rate_by_internet_service_types_average_download_query = """
SELECT InternetService, InternetType, AvgMonthlyGBDownload, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY InternetService, InternetType, AvgMonthlyGBDownload
ORDER BY ChurnRate DESC
"""
churn_rate_by_internet_service_types_average_download_result = pd.read_sql_query(churn_rate_by_internet_service_types_average_download_query, conn)
churn_rate_by_internet_service_types_average_download_result

,InternetService,InternetType,AvgMonthlyGBDownload,ChurnCounts,ChurnRate
0,No,None,0,113,6.046014
1,Yes,Fiber Optic,27,56,2.996255
2,Yes,Fiber Optic,23,50,2.675227
3,Yes,Fiber Optic,11,45,2.407705
4,Yes,Fiber Optic,26,43,2.300696
...,...,...,...,...,...
132,Yes,DSL,56,1,0.053505
133,Yes,DSL,71,1,0.053505
134,Yes,Fiber Optic,39,1,0.053505
135,Yes,Fiber Optic,56,1,0.053505


# Calculate Q3 churn rate based on customers' online security for Q3
# 
This query evaluates the churn rate among customers based on their use of online security features, helping to determine if security options influence customer loyalty.

In [25]:
# 23. Churn Rate Based on Online Security for Q3
churn_rate_based_on_online_security_query = """
SELECT OnlineSecurity, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY OnlineSecurity
ORDER BY ChurnRate DESC
"""
churn_rate_based_on_online_security_result = pd.read_sql_query(churn_rate_based_on_online_security_query, conn)
churn_rate_based_on_online_security_result

,OnlineSecurity,ChurnCounts,ChurnRate
0,No,1574,84.216158
1,Yes,295,15.783842


# Calculate churn rate by online security, online backup, device protection, and premium tech support for Q3

This query assesses how various support features, such as online security, backup, device protection, and premium tech support, impact churn rates, indicating the importance of customer support services.

In [26]:
# 24. Churn Rate by Online Security, Online Backup, Device Protection, and Premium Tech Support for Q3
churn_rate_by_security_backup_device_support_query = """
SELECT OnlineSecurity, OnlineBackup, DeviceProtectionPlan, PremiumTechSupport, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY OnlineSecurity, OnlineBackup, DeviceProtectionPlan, PremiumTechSupport
ORDER BY ChurnRate DESC
"""
churn_rate_by_security_backup_device_support_result = pd.read_sql_query(churn_rate_by_security_backup_device_support_query, conn)
churn_rate_by_security_backup_device_support_result

,OnlineSecurity,OnlineBackup,DeviceProtectionPlan,PremiumTechSupport,ChurnCounts,ChurnRate
0,No,No,No,No,831,44.462279
1,No,No,Yes,No,207,11.075441
2,No,Yes,No,No,196,10.486891
3,No,Yes,Yes,No,129,6.902087
4,Yes,No,No,No,92,4.922418
5,No,No,No,Yes,75,4.012841
6,No,No,Yes,Yes,57,3.049759
7,Yes,Yes,No,No,42,2.247191
8,No,Yes,No,Yes,40,2.140182
9,No,Yes,Yes,Yes,39,2.086677


# Calculate churn rate by streaming TV, streaming movies, and streaming music for Q3

This query analyzes the churn rate based on customers' subscriptions to streaming TV, movies, and music services, providing insights into how entertainment options affect customer retention.

In [27]:
# 25. Churn Rate by Streaming TV, Streaming Movies, and Streaming Music for Q3
churn_rate_by_streaming_services_query = """
SELECT StreamingTV, StreamingMovies, StreamingMusic, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY StreamingTV, StreamingMovies, StreamingMusic
ORDER BY ChurnRate DESC
"""
churn_rate_by_streaming_services_result = pd.read_sql_query(churn_rate_by_streaming_services_query, conn)
churn_rate_by_streaming_services_result

,StreamingTV,StreamingMovies,StreamingMusic,ChurnCounts,ChurnRate
0,No,No,No,744,39.807384
1,Yes,Yes,Yes,450,24.077047
2,Yes,No,No,220,11.771001
3,No,Yes,Yes,191,10.219369
4,Yes,Yes,No,121,6.474050
5,No,No,Yes,64,3.424291
6,No,Yes,No,56,2.996255
7,Yes,No,Yes,23,1.230605


# Calculate churn rate by contract

This query evaluates the churn rate across different types of customer contracts, which can indicate how contract terms relate to customer retention.

In [28]:
# 26. Churn Rate by Contract for Q3
churn_rate_by_contract_query = """
SELECT Contract, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY Contract
ORDER BY ChurnRate DESC
"""
churn_rate_by_contract_result = pd.read_sql_query(churn_rate_by_contract_query, conn)
churn_rate_by_contract_result

,Contract,ChurnCounts,ChurnRate
0,Month-to-Month,1655,88.550027
1,One Year,166,8.881755
2,Two Year,48,2.568218


# Calculate churn rate by payment method # 
This query analyzes the churn rate for customers based on their chosen payment methods, which may reveal preferences or issues related to billing.

In [29]:
# 27. Churn Rate by Payment Method for Q3
churn_rate_by_payment_method_query = """
SELECT PaymentMethod, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY PaymentMethod
ORDER BY ChurnRate DESC
"""
churn_rate_by_payment_method_result = pd.read_sql_query(churn_rate_by_payment_method_query, conn)
churn_rate_by_payment_method_result

,PaymentMethod,ChurnCounts,ChurnRate
0,Bank Withdrawal,1329,71.107544
1,Credit Card,398,21.294810
2,Mailed Check,142,7.597646


# Calculate churn rate by monthly charges

This query segments customers by their monthly charges and computes the churn rate for each charge segment, helping to identify if pricing influences churn.

In [30]:
# 28. Churn Rate by Monthly Charges
monthly_charges_churn_rate_query = """
WITH Churned AS (
    SELECT 
        CASE
            WHEN MonthlyCharge BETWEEN 0 AND 25 THEN '0 - 25'
            WHEN MonthlyCharge BETWEEN 26 AND 50 THEN '26 - 50'
            WHEN MonthlyCharge BETWEEN 51 AND 75 THEN '51 - 75'
            ELSE '76+'
        END AS MonthlyCharges, COUNT(*) AS ChurnCounts
    FROM TelcoCustomerChurn
    WHERE CustomerStatus = 'Churned'
    GROUP BY 
        CASE
            WHEN MonthlyCharge BETWEEN 0 AND 25 THEN '0 - 25'
            WHEN MonthlyCharge BETWEEN 26 AND 50 THEN '26 - 50'
            WHEN MonthlyCharge BETWEEN 51 AND 75 THEN '51 - 75'
            ELSE '76+'
        END
)
SELECT MonthlyCharges, ChurnCounts, 
       (ChurnCounts * 1.0 / SUM(ChurnCounts) OVER ()) * 100 AS ChurnRate  
FROM Churned
ORDER BY ChurnRate DESC
"""
monthly_charges_churn_rate_result = pd.read_sql_query(monthly_charges_churn_rate_query, conn)
monthly_charges_churn_rate_result

,MonthlyCharges,ChurnCounts,ChurnRate
0,76+,1136,60.781166
1,51 - 75,393,21.027287
2,26 - 50,213,11.396469
3,0 - 25,127,6.795078


# Calculate churn rate by total revenue

This query segments customers by total revenue generated and calculates the corresponding churn rate for each segment, providing insights into revenue impact on customer retention.

In [31]:
# 29. Churn Rate by Total Revenue
total_revenue_churn_rate_query = """
WITH Churned AS (
    SELECT 
        CASE
            WHEN TotalRevenue BETWEEN 0 AND 1000 THEN '0 - 1000'
            WHEN TotalRevenue BETWEEN 1001 AND 3000 THEN '1001 - 3000'
            WHEN TotalRevenue BETWEEN 3001 AND 5000 THEN '3001 - 5000'
            ELSE '5000+'
        END AS TotalRevenue, COUNT(*) AS ChurnCounts
    FROM TelcoCustomerChurn
    WHERE CustomerStatus = 'Churned'
    GROUP BY 
        CASE
            WHEN TotalRevenue BETWEEN 0 AND 1000 THEN '0 - 1000'
            WHEN TotalRevenue BETWEEN 1001 AND 3000 THEN '1001 - 3000'
            WHEN TotalRevenue BETWEEN 3001 AND 5000 THEN '3001 - 5000'
            ELSE '5000+'
        END
)
SELECT TotalRevenue, ChurnCounts, 
       (ChurnCounts * 1.0 / SUM(ChurnCounts) OVER ()) * 100 AS ChurnRate  
FROM Churned
ORDER BY ChurnRate DESC
"""
total_revenue_churn_rate_result = pd.read_sql_query(total_revenue_churn_rate_query, conn)
total_revenue_churn_rate_result

,TotalRevenue,ChurnCounts,ChurnRate
0,0 - 1000,978,52.327448
1,1001 - 3000,432,23.113965
2,5000+,247,13.215623
3,3001 - 5000,212,11.342964


# Calculate churn rate by satisfaction score for Q3

This query retrieves the count of customers and their churn rates based on satisfaction scores, which can indicate the correlation between satisfaction and churn behavior.

In [32]:
# 30. Churn Rate by Satisfaction Score for Q3
satisfaction_score_churn_rate_query = """
SELECT SatisfactionScore, COUNT(*) AS CustomerCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS Percentage
FROM TelcoCustomerChurn
GROUP BY SatisfactionScore
ORDER BY Percentage DESC
"""
satisfaction_score_churn_rate_result = pd.read_sql_query(satisfaction_score_churn_rate_query, conn)
satisfaction_score_churn_rate_result

,SatisfactionScore,CustomerCounts,Percentage
0,3,2665,37.838989
1,4,1789,25.401107
2,5,1149,16.314071
3,1,922,13.091012
4,2,518,7.354820


In [33]:
satisfaction_score_churn_rate_query = """
SELECT SatisfactionScore, COUNT(*) AS ChurnCounts, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
GROUP BY SatisfactionScore
ORDER BY ChurnRate DESC
"""
satisfaction_score_churn_rate_result = pd.read_sql_query(satisfaction_score_churn_rate_query, conn)
satisfaction_score_churn_rate_result

,SatisfactionScore,ChurnCounts,ChurnRate
0,1,922,49.331193
1,2,518,27.715356
2,3,429,22.953451


# Calculate churn rate by customer status

This query calculates the distribution of customers by their status (Churned vs. Not Churned) and computes the churn rate, allowing for assessment of overall customer health.

In [34]:
# 31. Churn Rate by Customer Status
customer_status_rate_query = """
SELECT CustomerStatus, COUNT(*) AS CustomerCounts, 
       (COUNT(*) * 1.0 / (SELECT COUNT(*) FROM TelcoCustomerChurn)) * 100 AS Rate
FROM TelcoCustomerChurn
GROUP BY CustomerStatus
ORDER BY CustomerCounts DESC
"""
customer_status_rate_result = pd.read_sql_query(customer_status_rate_query, conn)
customer_status_rate_result

,CustomerStatus,CustomerCounts,Rate
0,Stayed,4720,67.016896
1,Churned,1869,26.536987
2,Joined,454,6.446117


# Calculate average monthly charge by customer status

This query computes the average monthly charge incurred by customers, grouped by their status, which may inform strategies to reduce churn through pricing adjustments.

In [35]:
# 32. Average Monthly Charge by Customer Status
average_monthly_charge_status_query = """
SELECT CustomerStatus, AVG(MonthlyCharge) AS AvgMonthlyCharge
FROM TelcoCustomerChurn
GROUP BY CustomerStatus
"""
average_monthly_charge_status_result = pd.read_sql_query(average_monthly_charge_status_query, conn)
average_monthly_charge_status_result

,CustomerStatus,AvgMonthlyCharge
0,Churned,74.441332
1,Joined,43.474009
2,Stayed,62.976388


# Calculate churn rate by churn category

This query analyzes the churn rate based on predefined churn categories, showing how many customers fall into each category and what might be causing them to leave.

In [36]:
# 33. Churn Rate by Churn Category
churn_category_rate_query = """
SELECT ChurnCategory, COUNT(*) AS Count, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE ChurnCategory IS NOT NULL
GROUP BY ChurnCategory
ORDER BY Count DESC
"""
churn_category_rate_result = pd.read_sql_query(churn_category_rate_query, conn)
churn_category_rate_result

,ChurnCategory,Count,ChurnRate
0,Competitor,841,44.997325
1,Attitude,314,16.800428
2,Dissatisfaction,303,16.211878
3,Price,211,11.289460
4,Other,200,10.700910


# Calculate churn rate by churn reason

This query assesses the churn rate according to the reasons provided by customers for leaving, which can guide improvement efforts in service offerings.

In [37]:
# 34. Churn Rate by Churn Reason
churn_reason_rate_query = """
SELECT ChurnReason, COUNT(*) AS Count, 
       (COUNT(*) * 1.0 / SUM(COUNT(*)) OVER ()) * 100 AS ChurnRate
FROM TelcoCustomerChurn
WHERE ChurnReason IS NOT NULL
GROUP BY ChurnReason
ORDER BY Count DESC
"""
churn_reason_rate_result = pd.read_sql_query(churn_reason_rate_query, conn)
churn_reason_rate_result

,ChurnReason,Count,ChurnRate
0,Competitor had better devices,313,16.746923
1,Competitor made better offer,311,16.639914
2,Attitude of support person,220,11.771001
3,Don't know,130,6.955591
4,Competitor offered more data,117,6.260032
5,Competitor offered higher download speeds,100,5.350455
6,Attitude of service provider,94,5.029428
7,Price too high,78,4.173355
8,Product dissatisfaction,77,4.119850
9,Network reliability,72,3.852327


# Churn rate by CLTV segment for Q3

This query segments customers based on their Customer Lifetime Value (CLTV) and calculates churn rates within these segments, indicating which segments are more at risk.

In [38]:
# 35. Churn Rate by CLTV Segment for Q3
cltv_segment_query = """
WITH CLTVSegments AS (
    SELECT 
        CASE 
            WHEN CLTV < 1000 THEN 'Low Value'
            WHEN CLTV BETWEEN 1000 AND 5000 THEN 'Mid Value'
            ELSE 'High Value'
        END AS CLTVSegment,
        COUNT(*) AS TotalCustomers,
        SUM(CASE WHEN CustomerStatus = 'Churned' THEN 1 ELSE 0 END) AS ChurnCounts,
        AVG(CLTV) AS AvgCLTV
    FROM TelcoCustomerChurn
    GROUP BY 
        CASE 
            WHEN CLTV < 1000 THEN 'Low Value'
            WHEN CLTV BETWEEN 1000 AND 5000 THEN 'Mid Value'
            ELSE 'High Value'
        END
)
SELECT 
    CLTVSegment, 
    ChurnCounts, 
    TotalCustomers,
    (ChurnCounts * 1.0 / TotalCustomers) * 100 AS ChurnRate, 
    AvgCLTV 
FROM CLTVSegments
"""
cltv_segment_result = pd.read_sql_query(cltv_segment_query, conn)
cltv_segment_result

,CLTVSegment,ChurnCounts,TotalCustomers,ChurnRate,AvgCLTV
0,High Value,553,2572,21.500778,5620.531882
1,Mid Value,1316,4471,29.434131,3698.339298


# Calculate overall retention rate

This query computes the overall retention rate of customers, showing the percentage that have not churned, which is crucial for assessing business health.

In [39]:
# 36. Overall Retention Rate
overall_retention_rate_query = """
SELECT (COUNT(CASE WHEN CustomerStatus <> 'Churned' THEN 1 END) * 1.0 / COUNT(*)) * 100 AS RetentionRate
FROM TelcoCustomerChurn
"""
overall_retention_rate_result = pd.read_sql_query(overall_retention_rate_query, conn)
overall_retention_rate_result

,RetentionRate
0,73.463013


# Calculate total revenue lost due to churned customers

This query calculates the total revenue lost from customers who have churned, providing insight into the financial impact of churn and guiding strategic decisions.

In [40]:
# 37. Total Revenue Lost Due to Churned Customers
total_revenue_lost_query = """
SELECT SUM(TotalRevenue) AS TotalRevenueLost
FROM TelcoCustomerChurn
WHERE CustomerStatus = 'Churned'
"""
total_revenue_lost_result = pd.read_sql_query(total_revenue_lost_query, conn)
total_revenue_lost_result

,TotalRevenueLost
0,3684459.82


In [41]:
# Close the database connection
conn.close()